# Main particulars Optiwise

In [1]:
#import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
import numpy as np
import yaml

from vessel_manoeuvring_models.visualization.plot import track_plot, plot
from phd.visualization.plot_ship import track_plots
from vessel_manoeuvring_models.prime_system import PrimeSystem
from phd.visualization.plot_prediction import plot_total_force, plot_force_components, predict, plot_compare_model_forces, plot_parameter_contributions, group_parameters, joined_parameter_contributions, same_ylims
from vessel_manoeuvring_models.symbols import *
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify, remove_functions, prime
from vessel_manoeuvring_models.parameters import df_parameters as df_parameters_all
p = df_parameters_all["symbol"]
import statsmodels.api as sm
import sympy as sp
from sympy import Eq, symbols, latex
import paper
from paper import save_fig, save_eq
import yaml
#with open("styles.yml", mode='r') as file:
#    styles = yaml.safe_load(file)
from phd.visualization.styles import styles
from phd.pipelines.predict.nodes import score
import vessel_manoeuvring_models.models.IMO_simulations
from phd.paper.equation import to_latex

In [3]:
model_wPCC = catalog.load(f"wPCC.models_VCT_MDL")['semiempirical_covered_inertia']()

[01/27/25 13:32:50] WARNING  /home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/pa ]8;id=173568;file:///home/maa/miniconda3/envs/phd/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=601239;file:///home/maa/miniconda3/envs/phd/lib/python3.10/warnings.py#109\109]8;;\
                             rtitioned_dataset.py:200: KedroDeprecationWarning:                                    
                             'PartitionedDataset' has been moved to `kedro-datasets` and will be                   
                             removed in Kedro 0.19.0.                                                              
                               warnings.warn(                                                                      
                                                                                                                   

                    WARNING  /home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/__ ]8;id=889240;file:///home/maa/miniconda3/envs/phd/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=856594;file:///home/maa/miniconda3/envs/phd/lib/python3.10/warnings.py#109\109]8;;\
                             init__.py:44: KedroDeprecationWarning: 'AbstractDataSet' has been                     
                             renamed to 'AbstractDataset', and the alias will be removed in Kedro                  
                             0.19.0                                                                                
                               return getattr(kedro.io.core, name)                                                 
                                                                                                                   

                    INFO     Loading data from 'wPCC.models_VCT_MDL' (PartitionedDataset)...    ]8;id=556952;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=441176;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [4]:
#%reload_kedro
ship='optiwise'
ship_data = catalog.load(f"{ship}.ship_data")

loaders = {
    "semiempirical rudder" : catalog.load(f"{ship}.models_VCT_MDL")['semiempirical_covered_inertia'],
}

[01/27/25 13:33:11] INFO     Loading data from 'optiwise.ship_data' (YAMLDataSet)...            ]8;id=60440;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=712307;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'optiwise.models_VCT_MDL'                        ]8;id=95150;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=122204;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\
                             (PartitionedDataset)...                                                               

In [5]:
catalog.load(f"{ship}.models_VCT_MDL")

                    INFO     Loading data from 'optiwise.models_VCT_MDL'                        ]8;id=813051;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=206225;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\
                             (PartitionedDataset)...                                                               


{
    'semiempirical_covered_inertia': <bound method AbstractDataset.load of <phd.extras.datasets.model_dataset.ModularVesselSimulatorDataSet object at 0x7ff53f797100>>
}

In [6]:
models = {name:loader() for name,loader in loaders.items()}

In [7]:
dofs=['X','Y','N']
_=[]
for model_name,model in models.items():
    pars_all = {}
    for dof in dofs:
        pars = {key:value for key,value in model.parameters.items() if ((value!=0) and (key[0]==dof) and (not 'dot' in key) and (not 'delta' in key) and (not 'thrust' in key))}
        pars_all.update(pars)
    
    pars_all['model'] = styles[model_name]['label']
    _.append(pars_all)

df_parameters = pd.DataFrame(_)
df_parameters = df_parameters.set_index('model').transpose()
df_parameters.index = pd.Series(df_parameters.index).apply(lambda x:fr"${{{x[0]}'}}_{{{x[1:]}}}$")
df_parameters.index.name='Coefficient'

In [8]:
df_parameters

model,semi-empirical rudder
Coefficient,
${X'}_{0}$,-0.001363
${X'}_{u}$,0.000417
${X'}_{vv}$,-0.002161
${X'}_{rr}$,-0.001232
${X'}_{vr}$,-0.005647
${Y'}_{vvv}$,-0.176322
${Y'}_{v}$,-0.015159
${Y'}_{r}$,0.006785
${Y'}_{vrr}$,-0.105037


In [9]:
df_parameters.to_csv(paper.file_path_with_nb_ref("parameters.csv", directory='tables'), index=True)

## Other parameters

In [10]:
keys = list(pars_all.keys())
keys.remove('model')
keys


[
    'X0',
    'Xu',
    'Xvv',
    'Xrr',
    'Xvr',
    'Yvvv',
    'Yv',
    'Yr',
    'Yvrr',
    'Yvvr',
    'Nv',
    'Nvvv',
    'Nr',
    'Nvvr',
    'Nvrr'
]

In [11]:
model = models["semiempirical rudder"]
other_parameters = pd.Series(model.parameters)
other_parameters = other_parameters.drop(keys);

## Added masses

In [12]:
mask = other_parameters.index.str.contains('dot')
added_masses = other_parameters[mask]
other_parameters = other_parameters.drop(added_masses.index);
added_masses


Xudot   -0.001864
Yvdot   -0.015473
Yrdot   -0.000537
Nvdot   -0.000537
Nrdot   -0.000727
dtype: float64

In [13]:
pd.DataFrame(added_masses).transpose().to_csv(paper.file_path_with_nb_ref("added_masses.csv", directory='tables'), index=False)

In [14]:
df_added_masses = pd.DataFrame(index=added_masses.index)
#df_added_masses['Parameter'] = df_parameters_all.loc[added_masses.index]['symbol']
#df_added_masses['Parameter'] = df_added_masses['Parameter'].apply(lambda x: f"${x}$")
df_added_masses['Value'] = added_masses
#df_added_masses = df_added_masses.set_index('Parameter').transpose()
df_added_masses = df_added_masses.transpose()

In [15]:
df_added_masses

,Xudot,Yvdot,Yrdot,Nvdot,Nrdot
Value,-0.001864,-0.015473,-0.000537,-0.000537,-0.000727


In [16]:
df_added_masses.to_csv(paper.file_path_with_nb_ref("added_masses.csv", directory='tables'), index=False)

In [17]:
symbols = []
for key, eq in model.equations.items():
    symbols+=list(eq.rhs.free_symbols)
symbols = list(set(symbols))

In [138]:
parameters = {}
for symbol in symbols:
    key = str(symbol)
    if key in model.parameters:
        parameters[symbol] = model.parameters[key]  

    if key in model.ship_parameters:
        parameters[symbol] = model.ship_parameters[key]    

In [139]:
parameters


{
    kappa_v_pos: 0.7999995368183748,
    Omega: 0,
    kappa_r_neg: 0.6954713902464055,
    C_2_beta_p_neg: 1.1,
    x_H: 0.2110586462690569,
    x_R: -2.389705882205995,
    s: 0,
    A_R_C: 0.02506725778238429,
    kappa_r_pos: 0.6954713902464055,
    C_D_tune: 1.322683954360055,
    kappa_v_neg: 0.2666665122727916,
    A_R_U: 0.009695826123375053,
    rho: 1026.0,
    a_H: 0.27814404918002955,
    e_0: 0.9,
    A_R: 0.03476308390575934,
    C_2_beta_p_pos: 1.6,
    C_D0_tune: 8.56397840233827,
    C_1: 2.0,
    c_r: 0.17161764704827054,
    C_L_tune: 1,
    y_R: 0,
    x_p: -2.3036764704465793,
    nu: 1.18849e-06,
    r_0: 0.07794117646579553,
    gamma_0: -0.023212879051524585,
    w_p0: 0.5049,
    c_t: 0.14999999999077632,
    b_R: 0.2161764705749423,
    delta_lim: 0.6108652381980153,
    x: 0.08602941175941581
}

In [134]:
df_parameters = pd.DataFrame(index=parameters.keys())
df_parameters['Parameter'] = pd.Series(df_parameters.index, index=df_parameters.index).apply(lambda x : f"${to_latex(x)}$")
df_parameters.sort_values(by='Parameter', inplace=True)
df_parameters['Value'] = parameters
df_parameters.index = pd.Series(df_parameters.index).astype(str)

#df_parameters.drop(index=['delta_alpha_s'], inplace=True)  # remove stalling
df_parameters.drop(index=['y_R'], inplace=True)  # remove stalling
df_parameters.drop(index=['C_L_tune'], inplace=True)  # not used
#df_parameters.reset_index(inplace=True, drop=True)

In [132]:
df_parameters

,Parameter,Value
A_R_C,$A_{R C}$,0.025067
A_R_U,$A_{R U}$,0.009696
A_R,$A_{R}$,0.034763
C_1,$C_{1}$,2.000000
C_2_beta_p_neg,$C_{2 \beta p neg}$,1.100000
C_2_beta_p_pos,$C_{2 \beta p pos}$,1.600000
C_D_tune,$C_{D tune}$,1.322684
C_D0_tune,$C_{D0 tune}$,8.563978
Omega,$\Omega$,0.000000
delta_lim,$\delta_{lim}$,0.610865


In [96]:
df_parameters.loc['delta_lim','Parameter'] = r'$\delta_{lim}$ $[rad]$'
df_parameters.loc['gamma_0_port','Parameter'] = r'$\gamma_{0port}$ $[rad]$'
df_parameters.loc['gamma_0_stbd','Parameter'] = r'$\gamma_{0stbd}$ $[rad]$'

In [97]:
table = paper.table_columns(df_parameters,columns=3)

In [98]:
table

,Parameter,Value,Parameter1,Value1,Parameter2,Value2
0,$A_{R C}$,0.025067,$\kappa_{r neg}$,0.695471,$r_{0}$,0.077941
1,$A_{R U}$,0.009696,$\kappa_{r pos}$,0.695471,$s$,0.000000
2,$A_{R}$,0.034763,$\kappa_{v neg}$,0.266667,$w_{p0}$,0.504900
3,$C_{1}$,2.000000,$\kappa_{v pos}$,0.800000,$x$,0.086029
4,$C_{2 \beta p neg}$,1.100000,$\nu$,0.000001,$x_{H}$,0.211059
5,$C_{2 \beta p pos}$,1.600000,$\rho$,1026.000000,$x_{R}$,-2.389706
6,$C_{D tune}$,1.322684,$a_{H}$,0.278144,$x_{p}$,-2.303676
7,$C_{D0 tune}$,8.563978,$b_{R}$,0.216176,$\gamma_{0port}$ $[rad]$,NaN
8,$\Omega$,0.000000,$c_{r}$,0.171618,$\gamma_{0stbd}$ $[rad]$,NaN
9,$\delta_{lim}$ $[rad]$,0.610865,$c_{t}$,0.150000,NaN,NaN


In [99]:
table.to_csv(paper.file_path_with_nb_ref("other_parameters.csv", directory='tables'), index=False)

## Main particulars joined

In [100]:
def ship_parameters_to_table(model):
    ship_parameters = pd.Series(model.ship_parameters)
    #keys = list(set(ship_parameters.index) - set(df_parameters.index))
    keys = list(set(ship_parameters.index))
    ship_parameters = ship_parameters[keys].copy()
    
    drops = [
        'TWIN',
        'y_fan_fore',
        'x_p',
        'y_R_stbd',
        'y_fan_aft',
        'x_r',
        'volume',
        'n_prop',
        'z_p',
        'x_fan_fore',
        'y_R_port',
        'A_YV',
        'A_XV',
        'rho_A',
        'x_fan_aft',
        'CB',
        'c',
        'w_p0',
        'z_R',
        'y_R',
        'y_p_port',
        'y_p_stbd',
        
    ]
    
    for key in drops:
        if key in ship_parameters:
            ship_parameters.drop(index=key, inplace=True)

    return ship_parameters

In [101]:
model.ship_parameters


{
    'A_R': 0.03476308390575934,
    'B': 0.8823529411222136,
    'CB': 0.7733293347328436,
    'D': 0.15588235293159106,
    'I_z': 1471.5096397140396,
    'L': 4.77941176441199,
    'T': 0.30147058821675626,
    'TWIN': 0.0,
    'b_R': 0.2161764705749423,
    'c_r': 0.17161764704827054,
    'c_t': 0.14999999999077632,
    'm': 1008.7268469541564,
    'rho': 1026.0,
    'scale_factor': 68.0000000041814,
    'tdf': 0.21,
    'volume': 0.9831645681814392,
    'x_G': 0.17636764704797847,
    'x_p': -2.3036764704465793,
    'x_r': -2.389705882205995,
    'y_R': 0,
    'y_p': 0.0,
    'r_0': 0.07794117646579553,
    'x': 0.08602941175941581,
    'x_R': -2.389705882205995,
    'z_R': 0,
    'A_R_C': 0.02506725778238429,
    'A_R_U': 0.009695826123375053
}

In [102]:
to_latex('x_R')

'\\mathtt{\\text{x\\_R}}'

In [103]:
ship_parameters_optiwise = ship_parameters_to_table(model=model)

In [104]:
ship_parameters_wPCC = ship_parameters_to_table(model=model_wPCC)

In [105]:
df_ship_parameters = pd.DataFrame(index=ship_parameters_wPCC.index)
df_ship_parameters['Parameter'] = pd.Series(df_ship_parameters.index, index=df_ship_parameters.index).apply(lambda x : f"${to_latex(x)}$")
df_ship_parameters['wPCC'] = ship_parameters_wPCC
df_ship_parameters['Optiwise'] = ship_parameters_optiwise
df_ship_parameters['Description'] = ''

df_ship_parameters.drop(index=[
    'A_R_C',
    'A_R_U',
    'r_0',
    'rho',
    'x',
    
], inplace=True)

In [106]:
df_ship_parameters.loc['I_z','Parameter'] = r'$I_{zz}$'
df_ship_parameters.loc['x_G','Parameter'] = r'$x_{G}$'
df_ship_parameters.loc['scale_factor','Parameter'] = r'Scale factor'
df_ship_parameters.loc['tdf','Parameter'] = r'$t_{df}$'
df_ship_parameters.loc['T','Parameter'] = r'$T_{m}$'
df_ship_parameters.sort_index(inplace=True)

In [148]:
df_ship_parameters.loc['B','Description'] = r'Breadth'
df_ship_parameters.loc['D','Description'] = r'Propeller diameter'
df_ship_parameters.loc['I_z','Description'] = r'Yaw moment of inertia around L/2'
df_ship_parameters.loc['L','Description'] = r'Length between perpendiculars'
df_ship_parameters.loc['T','Description'] = r'Mean draught'
df_ship_parameters.loc['m','Description'] = r'Ship mass'
df_ship_parameters.loc['tdf','Description'] = r'Thrust deduction factor'
df_ship_parameters.loc['x_G','Description'] = r'Longitudinal c.o.g.'
df_ship_parameters.loc['A_R','Description'] = r'Projected rudder area'
df_ship_parameters.loc['A_R','Parameter'] = r'$A_R$'
df_ship_parameters.loc['b_R','Description'] = r'Rudder height'
df_ship_parameters.loc['b_R','Parameter'] = r'$b_R$'
df_ship_parameters.loc['c_r','Description'] = r'Rudder root chord'
df_ship_parameters.loc['c_r','Parameter'] = r'$c_r$'
df_ship_parameters.loc['c_t','Description'] = r'Rudder tip chord'
df_ship_parameters.loc['c_t','Parameter'] = r'$c_t$'
df_ship_parameters.loc['x_R','Description'] = r'Rudder position from L/2'
df_ship_parameters.loc['x_R','Parameter'] =  r'$x_R$'

df_ship_parameters.loc['B',  'Unit'] = r'm'
df_ship_parameters.loc['D',  'Unit'] = r'm'
df_ship_parameters.loc['I_z','Unit'] = r'kgm\textsuperscript{2}'
df_ship_parameters.loc['L',  'Unit'] = r'm'
df_ship_parameters.loc['T',  'Unit'] = r'm'
df_ship_parameters.loc['m',  'Unit'] = r'kg'
df_ship_parameters.loc['tdf','Unit'] = ''
df_ship_parameters.loc['x_G','Unit'] = 'm'
df_ship_parameters.loc['A_R','Unit'] = r'm\textsuperscript{2}'
df_ship_parameters.loc['b_R','Unit'] = 'm'
df_ship_parameters.loc['c_r','Unit'] = 'm'
df_ship_parameters.loc['c_t','Unit'] = 'm'
df_ship_parameters.loc['x_R','Unit'] = 'm'


In [149]:
df_ship_parameters

,Parameter,wPCC,Optiwise,Description,Unit
A_R,$A_R$,0.030096,0.034763,Projected rudder area,m\textsuperscript{2}
B,$\mathtt{\text{B}}$,0.946602,0.882353,Breadth,m
D,$\mathtt{\text{D}}$,0.121359,0.155882,Propeller diameter,m
I_z,$I_{zz}$,742.046279,1471.509640,Yaw moment of inertia around L/2,kgm\textsuperscript{2}
L,$\mathtt{\text{L}}$,5.014563,4.779412,Length between perpendiculars,m
T,$T_{m}$,0.206311,0.301471,Mean draught,m
b_R,$b_R$,0.174272,0.216176,Rudder height,m
c_r,$c_r$,0.192476,0.171618,Rudder root chord,m
c_t,$c_t$,0.152913,0.150000,Rudder tip chord,m
m,$\mathtt{\text{m}}$,441.026784,1008.726847,Ship mass,kg


In [150]:
to_latex('A_R')

'\\mathtt{\\text{A\\_R}}'

In [151]:
latex('A_{R}')

'\\mathtt{\\text{A\\_\\{R\\}}}'

In [152]:
df_ship_parameters.to_csv(paper.file_path_with_nb_ref("main_particulars.csv", directory='tables'), index=False)

In [34]:
model.ship_parameters


{
    'A_R': 0.03476308390575934,
    'B': 0.8823529411222136,
    'CB': 0.7733293347328436,
    'D': 0.15588235293159106,
    'I_z': 1471.5096397140396,
    'L': 4.77941176441199,
    'T': 0.30147058821675626,
    'TWIN': 0.0,
    'b_R': 0.2161764705749423,
    'c_r': 0.17161764704827054,
    'c_t': 0.14999999999077632,
    'm': 1008.7268469541564,
    'rho': 1026.0,
    'scale_factor': 68.0000000041814,
    'tdf': 0.21,
    'volume': 0.9831645681814392,
    'x_G': 0.17636764704797847,
    'x_p': -2.3036764704465793,
    'x_r': -2.389705882205995,
    'y_R': 0,
    'y_p': 0.0,
    'r_0': 0.07794117646579553,
    'x': 0.08602941175941581,
    'x_R': -2.389705882205995,
    'z_R': 0,
    'A_R_C': 0.02506725778238429,
    'A_R_U': 0.009695826123375053
}